In [1]:
import numpy as np
import pylab as plt
from math import *
from scipy import integrate
from cyfuncs import *
from exceptions import Exception, ValueError, OverflowError, ZeroDivisionError
%matplotlib inline

In [51]:
D = 40.
r0 = 0.35
rt = 2.#np.inf

theta = 0.5
Dprime = D/r0
rtprime = rt/r0
ymin = cos( np.radians(theta) )

def density(x):
    return zhao_func(x, 1., 3., 0.)

def radius(z, y, Dprime):
    try:
        return sqrt( z*z + Dprime*Dprime*(1-y*y) )
    except (OverflowError, ZeroDivisionError):
        return np.nan

In [52]:
def integrand(z, y, ymin, rc):
    try:
        return density( radius(z, y, Dprime) )**2
    except (OverflowError, ZeroDivisionError):
        return np.nan

rc = 0.001

def lim_z1(y, ymin, rc):
    alpha = atan( rc/D )
    A = cos( np.radians(alpha) )
    l1_min = D*y - sqrt( rt*rt - D*D* (1. - y*y) )
    l1_max = D*y if y<A else D*A - sqrt( rc*rc - D*D * (1. - A*A) )
    return [l1_min, l1_max]

def lim_z2(y, ymin, rc):
    alpha = atan( rc/D )
    A = cos( np.radians(alpha) )
    l2_min = D*y if y<A else D*A + sqrt( rc*rc - D*D * (1. - A*A) )
    l2_max = D*y + sqrt( rt*rt - D*D * (1. - y*y) )
    return [l2_min, l2_max]

def lim_z3(A, ymin, rc):
    l3_min = D*A - sqrt( rc*rc - D*D * (1. - A*A) )
    l3_max = D*A + sqrt( rc*rc - D*D * (1. - A*A) )
    return [l3_min, l3_max]

def lim_y(ymin, rc):
    return [ymin, 1.]

In [53]:
for i in range(-1, -10, -1):
    I1 = integrate.nquad(integrand, ranges=[lim_z1, lim_y], args=(ymin, 10**i), \
                    opts=[{'limit':1000, 'epsabs':1.e-10, 'epsrel':1.e-10},\
                          {'limit':1000, 'epsabs':1.e-10, 'epsrel':1.e-10}])
    
    print '%20.20g %20.20g'%I1

1.8492806996510759138e-14 5.4445525542679528284e-24
1.8492806996510759138e-14 5.4445525542679528284e-24
1.8492806996510759138e-14 5.4445525542679528284e-24
1.8492806996510759138e-14 5.4445525542679528284e-24
1.8492806996510759138e-14 5.4445525542679528284e-24
1.8492806996510759138e-14 5.4445525542679528284e-24
1.8492806996510759138e-14 5.4445525542679528284e-24
1.8492806996510759138e-14 5.4445525542679528284e-24
1.8492806996510759138e-14 5.4445525542679528284e-24


In [54]:
Amin = cos( np.radians( atan( rc/D ) ) )
I3 = integrate.nquad(integrand, ranges=[lim_z3, lim_y], args=(Amin, 0.01), \
                    opts=[{'limit':1000, 'epsabs':1.e-10, 'epsrel':1.e-10},\
                          {'limit':1000, 'epsabs':1.e-10, 'epsrel':1.e-10}])    

In [55]:
I3

(4.006062842650701e-25, 4.674522615263226e-26)

In [56]:
rc = 0.01
def integrand_y(y):
    return integrate.romberg(integrand, a=lim_z3(y, ymin, rc)[0], b=lim_z3(y, ymin, rc)[1], args=(y, ymin, rc), tol=1.e-10)

In [57]:
I1romb = integrate.romberg( integrand_y, a=lim_y(Amin, rc)[0], b=lim_y(Amin, rc)[1], tol=1.e-10)

In [58]:
I1romb

4.0060628414653311e-25